In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Adding all possible machine learning techniques, not going to use all.
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv
from sklearn.preprocessing import StandardScaler
from keras.layers.advanced_activations import ReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten,Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
import keras
import cv2
import tensorflow
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import plotly.express as px
import plotly.graph_objs as go
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix

In [ ]:
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
data.head()

In [ ]:
#import pandas_profiling
#from pandas_profiling import ProfileReport

#profile = ProfileReport(data, title='Pandas Profiling Report')
#profile

In [ ]:
import pandas as pd
import numpy as npo
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

As shown below time and ejection_fraction are strongly negatively correlated to Death_event.

In [ ]:
plt.subplots(figsize=(11, 11)) 
sns.heatmap(data.corr(),annot=True)

In [ ]:
#data=data.drop(['time'],axis=1)
#data=data.drop(['ejection_fraction'],axis=1)

In [ ]:
X=data.drop(['DEATH_EVENT'],axis=1)
y=data[['DEATH_EVENT']]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#preprocess
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#scaler.fit(data)
#scaled_features = scaler.transform(data)
#Convert to table format - MinMaxScaler
#df_MinMax = pd.DataFrame(data=scaled_features, columns=["age", "serum_creatinine","anaemia","creatinine_phosphokinase","diabetes","high_blood_pressure","platelets"
#                                                        ,"serum_sodium","time","ejection_fraction","sex","smoking","DEATH_EVENT"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

train_set, test_set = train_test_split(data, train_size=0.8, test_size=0.2, random_state = 42)
print(len(train_set), ' Training Set +', len(test_set), ' Testing Set')


In [ ]:
#Same for train set(Target is purchase)
X_train = train_set.drop("DEATH_EVENT", axis=1) # drop labels for training set
y_train = train_set["DEATH_EVENT"].copy()

#same for test set
X_test = test_set.drop("DEATH_EVENT", axis=1)
y_test = test_set["DEATH_EVENT"].copy()

# CATBOOST

from looking below upon the iterations, we can clearly narrow down the learning rate and iterations to get an accuracy score of 0.7833333333333333

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=48,learning_rate=0.315,depth=4)

cat.fit(X_train, y_train,eval_set=(X_test, y_test))

In [ ]:
from sklearn.metrics import accuracy_score
predict = cat.predict(X_test)
print(accuracy_score(y_test, predict))

Max-Min Normalization will be used as we will obtain smaller standard deviations which could further increase the accuracy, but proved not to increase it at all.

# XGBOOST 

In [ ]:
#load up xgboost
import xgboost as xgb

train = xgb.DMatrix(X_train, label = y_train)
test = xgb.DMatrix(X_test, label = y_test)

param = {'max_depth': 4, 'eta': 0.3, 'objective': 'multi:softmax', 'num_class':2}
param['nthread'] = 4
param['eval_metric'] = 'auc'

#epochs or iterations.
iters = 20
model = xgb.train(param, train, iters)

predictions = model.predict(test)
print(accuracy_score(y_test, predictions))